In [1]:
# Importing libraries
import numpy as np
from scipy.signal import stft, istft

In [6]:
# This will perform spectral subtraction on a noisy signal. Parameters: noisy signal, sampling frequency and alpha (for tuning)
# We will estimate the Power of the noise from the first few samples and subtract it from the signal with noise. 
# Alpha will determine how strong the subtraction is. Basically, we have trade-off between noise cancellation and speech signal degradation.
# We reconstruct the signal back to the time domain using the original phase.

def spectral_subtraction(signal_noisy, fs, alpha):

    f, t, complex_spectr = stft(signal_noisy, fs, nperseg = 1024, nfft = 1024, noverlap = 512, window = 'hann') # complex_spectr will also contain phase

    # Averaging across the first few samples as the voice is silent there, but noise exists 
    noise_mag = np.mean(np.abs(complex_spectr[:,:50]), axis = 1, keepdims = True) 

    # Subtracting noise from our original spectrum and preventing negative magnitudes
    mag = np.abs(complex_spectr) - alpha * noise_mag # Increasing alpha improves noise cancellation but degrades original speech quality a bit
    mag[mag < 1e-6] = 0 # This is done to kill the noise and avoid annoying clicking sounds when doing inverse stft
    
    # To Reconstruct, using the original signal phase is acceptable since estimating the exact phase is hard and our ears are less sensitive to phase
    phase = np.angle(complex_spectr)
    complex_spectr_denoised = mag * np.exp(1j * phase)

    # Taking Inverse short time fourier transform and clipping
    time, signal_denoised = istft(complex_spectr_denoised, fs, nperseg = 1024, nfft = 1024, noverlap = 512, window = 'hann')
    signal_denoised = np.clip(signal_denoised, -1, 1)

    return signal_denoised